# Import libraries

In [ ]:
%reset -f
# Mute warnings
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from category_encoders import MEstimateEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_predict

To know more about the dataset: https://www.kaggle.com/competitions/spaceship-titanic/dataa

# Data preprocessing

In [ ]:
def load_data():
    # Read data
    df_train = pd.read_csv("../input/spaceship-titanic/train.csv")
    df_test = pd.read_csv("../input/spaceship-titanic/test.csv")
    
    # Merge the splits so we can process them together
    df = pd.concat([df_train, df_test],ignore_index=True)
 
    # Preprocessing
    df = clean(df)
    df = impute(df)

    # Reform splits
    df_train = df[df.index<len(df_train)].reset_index(drop=True)
    df_test  = df[df.index >= len(df_train)].reset_index(drop=True)
    return df_train,df_test

## Cleaning data

In [ ]:
def clean(df):
    df[["gggg", "pp"]] = (df["PassengerId"].str.split("_", expand=True))
    df["gggg"] = df["gggg"].str.replace('^0+', '').astype(int)
    df["pp"] = df["pp"].str.replace('^0+', '').astype(int)
    df = df.drop(columns=["PassengerId","Name"])
    df[["deck", "num","side"]] = (  df["Cabin"].str.split("/", expand=True,n=3))
    df = df.drop(columns="Cabin")
    return df

## Imputing data

In [ ]:
def impute(df):
    cols_num = df.select_dtypes(include=np.number).columns.to_list()
    df[cols_num] = df[cols_num].fillna(0)
    df = df.fillna(df.mode().iloc[0])
    df["Transported"] = df["Transported"].astype(int)
    df["Age"] =  df["Age"].astype(int)
    return df

In [ ]:
def label_encode(X):
    cols_cat = X.select_dtypes(exclude=np.number).columns.to_list()
    for col in cols_cat:
        X[col] = X[col].astype('category')
        X[col] = X[col].cat.codes
    return X

# Load data

In [ ]:
df_train,df_test = load_data()

# Baseline

In [ ]:
def score_dataset(X, y, model=RandomForestClassifier()):
    cols_cat = X.select_dtypes(exclude=np.number).columns.to_list()
    for col in cols_cat:
        X[col] = X[col].astype('category')
        X[col] = X[col].cat.codes
    
    score = cross_val_score(
        model, X, y, cv=5, scoring="accuracy",
    )
   
    return score

In [ ]:
X = df_train.copy()
y = X.pop("Transported")

baseline_score = score_dataset(X, y)
baseline_score

# Feature Engineering

## Mutual information

In [ ]:
X = df_train.copy()
y = X.pop("Transported")
X = label_encode(X)
discrete_features = X.dtypes != float

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores  

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

Check for uninformative features

In [ ]:
def drop_uninformative(df, mi_scores,th=0):
    return df.loc[:, mi_scores > th]

In [ ]:
X = df_train.copy()
y = X.pop("Transported")
X = drop_uninformative(X, mi_scores)

score_dataset(X, y)

## Adding new features

In [ ]:
X = df_train.copy()
y = X.pop("Transported")
X = drop_uninformative(X, mi_scores)
X["total_exp"] = X["RoomService"] + X["FoodCourt"] +X["ShoppingMall"]+X["Spa"]+X["VRDeck"]
score_dataset(X, y)


We have improved peformances with our new feature

## Target encoding

Let's try target encoding for num 

In [ ]:
X = df_train.copy()

# Encoding split
X_encode = X.sample(frac=0.20, random_state=0)
y_encode = X_encode.pop("Transported")

# Training split
X_pretrain = X.drop(X_encode.index)
y_train = X_pretrain.pop("Transported")

encoder = MEstimateEncoder(
    cols=["deck"],
    m=1.0,
)

# Fit the encoder on the encoding split
encoder.fit(X_encode, y_encode)

# Encode the training split
X_train = encoder.transform(X_pretrain, y_train)

In [ ]:
feature = encoder.cols
plt.figure(dpi=90)
ax = sns.distplot(y_train, kde=True, hist=False)
ax = sns.distplot(X_train[feature], color='r', ax=ax, hist=True, kde=False, norm_hist=True)
ax.set_xlabel("Transported")

In [ ]:
X = df_train.copy()
y = X.pop("Transported")
score_base = score_dataset(X, y)
score_new = score_dataset(X_train, y_train)

print(score_base)
print(score_new)

In [ ]:
print("Previous score: ",score_new)
X_train["total_exp"] = X_train["RoomService"] + X_train["FoodCourt"] +X_train["ShoppingMall"]+X_train["Spa"]+X_train["VRDeck"]
X_train["ExpDeck"] = X_train.groupby("deck")["total_exp"].transform("mean")
X_train["ExpSide"] = X_train.groupby("side")["total_exp"].transform("mean")
X_train["ExpHome"] = X_train.groupby("HomePlanet")["total_exp"].transform("mean")
X_train["ExpCryo"] = X_train.groupby("CryoSleep")["total_exp"].transform("mean")
X_train["ExpDest"] = X_train.groupby("Destination")["total_exp"].transform("mean")
X_train["ExpVip"] = X_train.groupby("VIP")["total_exp"].transform("mean")
print("Score_group: ",score_dataset(X_train,y_train))

In [ ]:
X_train_norm = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
print("Score_group_norm",score_dataset(X_train_norm,y_train))

## KMEANS

In [ ]:

from sklearn.cluster import KMeans
def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    #X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new

def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    #X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)
    X_cd = kmeans.fit_transform(X_scaled)
    # Label features and join to dataset
    X_cd = pd.DataFrame(
        X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])]
    )
    return X_cd

In [ ]:
cluster_features = [
    "Age",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "total_exp"
]

In [ ]:
X_train_norm = X_train_norm.reset_index(drop=True)
X_train_norm =  X_train_norm.join(cluster_labels(X_train_norm, cluster_features, n_clusters=10))
print("Score_group_norm_clust",score_dataset(X_train_norm,y_train))

## PCA

In [ ]:

def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

def pca_components(df, features):
    X = df.loc[:, features]
    _, X_pca, _ = apply_pca(X)
    return X_pca

In [ ]:
pca_features = ['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'total_exp', 'ExpDeck', 'ExpSide', 'ExpHome', 'ExpCryo',
       'ExpDest', 'ExpVip']

In [ ]:
pca, X_pca, loadings = apply_pca(X_train_norm,standardize=False) #already normalized
plot_variance(pca)

In [ ]:
X_train_norm = X_train_norm.join(pca_components(X_train_norm,features=pca_features))
print("Score_group_norm_clust_pca",score_dataset(X_train_norm,y_train))

# Fine tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring="accuracy")
rf_random.fit(X_train_norm, y_train)

In [ ]:
rf_random.best_params_

Best pam
{'n_estimators': 400,\
 'min_samples_split': 10,\
 'min_samples_leaf': 4,\
 'max_features': 'sqrt',\
 'max_depth': 90,\
 'bootstrap': True}

In [ ]:
best_estimator = rf_random.best_estimator_

In [ ]:
import joblib
my_model = best_estimator
joblib.dump(my_model, "my_best_estimator.pkl")
my_model_loaded = joblib.load("my_best_estimator.pkl")

In [ ]:
preds = cross_val_predict(my_model_loaded,X_train_norm,y_train)
print("Best accuracy: ",np.sum(preds==y_train)/len(preds))

# TF model

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy',restore_best_weights = True,patience=10)

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=34),
    keras.layers.Dense(50,activation='relu'),
    keras.layers.Dense(50,activation='relu'),
    keras.layers.Dense(50,activation='relu'),
    keras.layers.Dense(50,activation='softmax')
])

model.compile(optimizer ='adam',loss=keras.losses.sparse_categorical_crossentropy,metrics=["accuracy"])
history = model.fit(X_train_norm,y_train,validation_split=0.1,verbose=True,epochs=50,callbacks=[callback])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
import matplotlib.pyplot as plt 
plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Submission

In [ ]:
id_test = pd.read_csv("../input/spaceship-titanic/test.csv")["PassengerId"]
X_test = df_test.drop(columns="Transported")

In [ ]:
#targed encoding
X_test = encoder.transform(X_test)

#Adding new features
X_test["total_exp"] = X_test["RoomService"] + X_test["FoodCourt"] +X_test["ShoppingMall"]+X_test["Spa"]+X_test["VRDeck"]
X_test["ExpDeck"] = X_test.groupby("deck")["total_exp"].transform("mean")
X_test["ExpSide"] = X_test.groupby("side")["total_exp"].transform("mean")
X_test["ExpHome"] = X_test.groupby("HomePlanet")["total_exp"].transform("mean")
X_test["ExpCryo"] = X_test.groupby("CryoSleep")["total_exp"].transform("mean")
X_test["ExpDest"] = X_test.groupby("Destination")["total_exp"].transform("mean")
X_test["ExpVip"] = X_test.groupby("VIP")["total_exp"].transform("mean")

#Clustering
X_test = label_encode(X_test)
X_test = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)
X_test = X_test.join(cluster_labels(X_test, cluster_features, n_clusters=10))

#Pca analysis
X_test = X_test.join(pca_components(X_test,features=pca_features))

In [ ]:
p = np.bool8(my_model_loaded.predict(X_test))
output = pd.DataFrame({'PassengerId': id_test, 'Transported': p})
output.to_csv('my_submission_.csv', index=False)
print("Your submission was successfully saved!")